In [6]:
import tempfile
import os
import calendar

In [7]:
def make_maps(aprx_fn, prism_folder, pdf_fn, first_month=1, last_month=12):
    temp_fn = os.path.join(tempfile.gettempdir(), 'temp.pdf')
    map_name = 'Temperature'
    layout_name = 'Temperature Layout'
    
    #Create variables containing the project, map, layout, and layer for later use
    project = arcpy.mp.ArcGISProject(aprx_fn)
    mapp = project.listMaps(map_name)[0]
    layout = project.listLayouts(layout_name)[0]
    layer = mapp.listLayers()[0]
    
    #Create PDF that will hold the final doc
    final_doc = arcpy.mp.PDFDocumentCreate(pdf_fn)

    #Loop through each month, add one to last month since range is not inclusive
    for month in range(first_month,last_month+1):
        #Variable to hold the month name
        month_n= calendar.month_name[month]
        print(f'Mapping {month_n}')
        #creating file name that references each month as we loop through
        month_fn = 'PRISM_tmean_stable_4kmM2_2017{0:0>2}_bil.bil'.format(month)

        #Update symbology by just changing the dataset that the layer uses
        #to the new dataset and keeping the existing symbology
        connection = layer.connectionProperties
        connection['dataset'] = month_fn
        layer.updateConnectionProperties(layer.connectionProperties, connection)

        #create variable holding the min and max values
        raster = arcpy.Raster(layer.dataSource)
        minn = int(round(raster.minimum))
        maxx = int(round(raster.maximum))

        #update title and statistics
        title = layout.listElements('TEXT_ELEMENT','title')[0]
        title.text = f'Mean annual temperature for {month_n}'

        statistics = layout.listElements('TEXT_ELEMENT','statistics')[0]
        statistics.text = f'Min: {minn}° C\nMax: {maxx}°C'

        #export to a temporary PDF
        layout.exportToPDF(temp_fn)

        #append temporary PDF to final PDF
        final_doc.appendPages(temp_fn)

    final_doc.saveAndClose()

    del final_doc, project

In [9]:
# Change these paths to match your computer!
aprx_fn = r'C:\python_GIS\week_10\mapbooks.aprx'
prism_folder = r'C:\python_GIS\week_10\PRISM_tmean_stable_4kmM2_2017_all_bil'

# Make a map book for all twelve months.
pdf_fn = r'C:\python_GIS\week_10\problem2a.pdf'
make_maps(aprx_fn, prism_folder, pdf_fn)

# Make a map book for March through July.
pdf_fn = r'C:\python_GIS\week_10\problem2b.pdf'
make_maps(aprx_fn, prism_folder, pdf_fn, 3, 7)

Mapping January
Mapping February
Mapping March
Mapping April
Mapping May
Mapping June
Mapping July
Mapping August
Mapping September
Mapping October
Mapping November
Mapping December
Mapping March
Mapping April
Mapping May
Mapping June
Mapping July
